<a href="https://colab.research.google.com/github/tasninkhanlamha/SkillMorph/blob/main/Assignment7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Load Dataset

```
**We will build a Model to detect thread mitigation by Blockchain consensus algorithm! **





In [97]:
# Imports and Setup
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib
matplotlib.use('Agg')  # Use non-interactive backend
import matplotlib.pyplot as plt
plt.switch_backend('Agg')
%matplotlib inline

import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, auc
from sklearn.inspection import permutation_importance
from sklearn.tree import DecisionTreeClassifier

sns.set(style='whitegrid')

print('Libraries imported and setup complete.')

Libraries imported and setup complete.


Libraries imported and setup complete.

In [98]:
# Load the dataset from the local directory
df = pd.read_csv('/content/drive/MyDrive/Skill Morph/iot_blockchain_security_dataset.csv', encoding='ascii', delimiter=',')
print('Dataset loaded with shape:', df.shape)
df.head()

Dataset loaded with shape: (1000, 11)


,Device ID,IoT Layer,Request Type,Data Size (KB),Processing Time (ms),Security Threat Type,Attack Severity (0-10),Blockchain Transaction Time (ms),Consensus Mechanism,Energy Consumption (mJ),Threat Mitigated
0,D0001,Application,Data Transmission,552,42,DDoS,1,142,PBFT,1.26,1
1,D0002,Application,Data Transmission,704,24,DDoS,1,197,PoS,0.66,0
2,D0003,Device,Authentication,485,35,Eavesdropping,7,272,PoS,0.92,0
3,D0004,Application,Data Transmission,965,21,Eavesdropping,1,187,PoS,1.88,0
4,D0005,Device,Encrypted Data Transfer,653,17,Tampering,1,155,PoS,1.80,1


Data Cleaning and Preprocessing¶
Let's quickly check the data for missing values and proper data types. Note that the column names carry some extra characters (e.g., units) so be mindful when referencing them.

In [99]:
# Check for missing values and data types
print('Missing values per column:')
print(df.isnull().sum())

print('\nData types:')
print(df.dtypes)

# If any cleaning is required (e.g., trimming whitespace or renaming columns), do it here
# For example, ensuring column names are stripped:
df.columns = [col.strip() for col in df.columns]# Verify changes
print('\nCleaned column names:')
print(df.columns.tolist())

Missing values per column:
Device ID                           0
IoT Layer                           0
Request Type                        0
Data Size (KB)                      0
Processing Time (ms)                0
Security Threat Type                0
Attack Severity (0-10)              0
Blockchain Transaction Time (ms)    0
Consensus Mechanism                 0
Energy Consumption (mJ)             0
Threat Mitigated                    0
dtype: int64

Data types:
Device ID                            object
IoT Layer                            object
Request Type                         object
Data Size (KB)                        int64
Processing Time (ms)                  int64
Security Threat Type                 object
Attack Severity (0-10)                int64
Blockchain Transaction Time (ms)      int64
Consensus Mechanism                  object
Energy Consumption (mJ)             float64
Threat Mitigated                      int64
dtype: object

Cleaned column names:
['Device

**Prepare Data**

In [100]:

# Convert categorical columns to numeric (e.g., Request Type, Security Threat Type, etc.)
df['Security Threat Type'] = df['Security Threat Type'].map({
    'DDoS': 1,
    'Eavesdropping': 2,
    'Tampering': 3,
    'Man-in-the-Middle': 4,
    'Unauthorized Access': 5,

})
df['Consensus Mechanism'] = df['Consensus Mechanism'].map({
    'PBFT': 1,
    'PoS': 2,
    'PoA': 3,
    'PoW': 4,

})
df['Request Type'] = df['Request Type'].map({
    'Data Transmission': 1,
    'Authentication': 2,
    'Smart Contract Execution': 3,
    'Encrypted Data Transfer': 4,

})


# Select the most important features
important_features = [
    'Data Size (KB)',            # Size of the data being transmitted
    'Processing Time (ms)',      # Time taken for processing the request
    'Attack Severity (0-10)',    # Severity of the attack
    'Blockchain Transaction Time (ms)', # Time taken for blockchain transaction
    'Energy Consumption (mJ)',   # Energy consumed during the request
    'Security Threat Type',     # Security threat type (now numeric)
    'Consensus Mechanism',      # Consensus mechanism (now numeric)
    'Request Type'              # Request type (now numeric)
]

# Create the feature matrix X (measurements of the IoT request details)
X = df[important_features]

# Use the numeric threat label as the target variable (y)
y = df['Threat Mitigated']

# Print shapes of X and y to verify
print(f"X (measurements): {X.shape}")
print(f"y (thread mitigated): {y.shape}")

X (measurements): (1000, 8)
y (thread mitigated): (1000,)


# **Train and Test Split**


In [101]:
# Split the dataset into training (80%) and testing (20%) sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=20)

# Print the sizes of the training and testing sets
print(f"Training: {len(X_train)} samples (computer will learn from these)")
print(f"Testing:  {len(X_test)} samples (computer will be tested on these)")

# Show some training examples (optional, to inspect a few rows)
print(f"\nTraining Examples (Computer learns from these):")
print(X_train.head())

Training: 800 samples (computer will learn from these)
Testing:  200 samples (computer will be tested on these)

Training Examples (Computer learns from these):
     Data Size (KB)  Processing Time (ms)  Attack Severity (0-10)  \
504             499                    33                       8   
839            1426                    44                       8   
141            1115                    45                       2   
825             734                    36                       8   
470             173                    41                       2   

     Blockchain Transaction Time (ms)  Energy Consumption (mJ)  \
504                               143                     1.29   
839                               119                     1.38   
141                               258                     1.31   
825                               236                     1.31   
470                               295                     1.34   

     Security Threat Type  

# **Train with Decision Tree**

In [102]:
# Create our Skill Morph model
#skillmorph_securityGuard = DecisionTreeClassifier(
 #   max_depth=1,  # Keep it simple
  #  random_state=20
#)

# Replace DecisionTreeClassifier with RandomForestClassifier
skillmorph_securityGuard = RandomForestClassifier(
    n_estimators=200,  # Increase number of trees
    max_depth=10,      # Set a max depth for trees
    min_samples_split=5,  # Minimum samples to split nodes
    min_samples_leaf=2,   # Minimum samples at each leaf
    max_features='sqrt',  # Number of features to consider
    random_state=20
  )

print("Teaching skillmorph_securityGuard  with training data...")
# Train the model
skillmorph_securityGuard.fit(X_train, y_train)
print("skillmorph_securityGuard training completed!")


Teaching skillmorph_securityGuard  with training data...
skillmorph_securityGuard training completed!


Test our skillmorph Security Guard

In [103]:

print("Now let's see how good our skillmorph_securityGuard is...")

# Make predictions
predictions = skillmorph_securityGuard.predict(X_test)

print("skillmorph_securityGuard made predictions for all test!")


# Calculate accuracy
accuracy = accuracy_score(y_test, predictions)
print(f"\nskillmorph_securityGuard Accuracy: {accuracy:.3f} ({accuracy*100:.1f}%)")

if accuracy > 0.95:
    print("EXCELLENT! Our skillmorph_securityGuard is very accurate!")
elif accuracy > 0.90:
    print("GOOD! Our skillmorph_securityGuardr performs well!")
elif accuracy > 0.80:
    print("OK! Our skillmorph_securityGuard is decent!")
else:
    print("Needs improvement...")



Now let's see how good our skillmorph_securityGuard is...
skillmorph_securityGuard made predictions for all test!

skillmorph_securityGuard Accuracy: 0.720 (72.0%)
Needs improvement...
